# Evaporation from CoAgMET data
*M. Vremec, University of Graz, 2021*

In this notebook it is shown how to compute (reference) evaporation ($E_0$) from meteorological data observed by the Colorado State University (CoAgMET) at Holyoke in Colorado, USA. The notebook also includes a comparison between $E_0$ estimated using *pyet* (FAO56) and $E_0$ available from CoAgMET. According to CoAgMET documentation, the provided reference evaporation is estimated using ASCE reference evaporation for short reference grass (Wright, 2000), which corresponds to the FAO-56 method used with *pyet*.

Source: https://coagmet.colostate.edu/station/selector

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyet as pyet
pyet.show_versions()

## 1. Load CoAgMET Data

In [ ]:
data = pd.read_csv("data/et_coagmet.txt", parse_dates=True, index_col="date")
data

In [ ]:
e0_coagmet = data["et_asce0"]

meteo = pd.DataFrame({"tmean":data["tavg"], 
                      "tmax":data["tmax"],
                      "tmin":data["tmin"], 
                      "rhmax":data["rhmax"]*100,
                      "rhmin":data["rhmin"]*100, 
                      "u2":data["windrun"]*1000/86400,
                      "rs":data["solar"]*86400/1000000})

tmean, tmax, tmin, rhmax, rhmin, wind, rs = [meteo[col] for col in meteo.columns]
lat = 40.49*np.pi/180

## 2. Comparison: pyet FAO56 vs CoAgMET ASCE 

In [ ]:
e0_fao56 = pyet.pm_fao56(tmean, wind, rs=rs, elevation=1138, lat=lat, 
                         tmax=tmax, tmin=tmin, rhmax=rhmax, rhmin=rhmin)

e0_fao56.plot()
e0_coagmet.plot();

## 3. Plot the results

We now plot the evaporation time series against each other to see how these compare. 

In [ ]:
plt.scatter(e0_fao56, e0_coagmet)
plt.plot([0,15],[0,15], color="red", label="1:1 line")
plt.legend()
plt.xlabel("E0 pyet-FAO56 [mm]")
plt.ylabel("E0 CoAgMET-ASCE [mm]");